<a href="https://colab.research.google.com/github/alxwxgx/repository/blob/main/lrc_loan_approval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
from google.colab import files
uploaded = files.upload()

Saving loan_approval.csv to loan_approval.csv


Loan Approval with Logistic Regression

In [38]:
import pandas as pd

In [75]:
# read the data
loan_data = pd.read_csv("loan_approval.csv")

In [8]:
# copy the data
loan_prep = loan_data.copy()

In [15]:
# preview the data
loan_prep.head(10)

,ch,income,loanamt,married_Yes,status_Y
1,1.0,4583,128.0,1,0
2,1.0,3000,66.0,1,1
3,1.0,2583,120.0,1,1
4,1.0,6000,141.0,0,1
5,1.0,5417,267.0,1,1
6,1.0,2333,95.0,1,1
7,0.0,3036,158.0,1,0
8,1.0,4006,168.0,1,1
9,1.0,12841,349.0,1,0
10,1.0,3200,70.0,1,1


In [6]:
# indentify the missing values
loan_prep.isna().sum(axis=0)

gender     13
married     3
ch         50
income      0
loanamt    22
status      0
dtype: int64

In [10]:
# drop the rows with missing values
loan_prep = loan_prep.dropna()

In [11]:
# drop the irrelevant columns -- gender
loan_prep = loan_prep.drop(['gender'], axis=1)

In [12]:
# create the dummy variables -- encodes the object cols to integer dtype
loan_prep.dtypes

married     object
ch         float64
income       int64
loanamt    float64
status      object
dtype: object

In [13]:
loan_prep = pd.get_dummies(loan_prep, drop_first=True)
loan_prep.head()

,ch,income,loanamt,married_Yes,status_Y
1,1.0,4583,128.0,1,0
2,1.0,3000,66.0,1,1
3,1.0,2583,120.0,1,1
4,1.0,6000,141.0,0,1
5,1.0,5417,267.0,1,1


In [19]:
# normalize the data -- converts 'loanamt' and 'income' to same scale at zero
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [20]:
loan_prep['income'] = scaler.fit_transform(loan_prep[['income']])
loan_prep['loanamt'] = scaler.fit_transform(loan_prep[['loanamt']])

In [21]:
loan_prep.head()

,ch,income,loanamt,married_Yes,status_Y
1,1.0,-0.128073,-0.194250,1,0
2,1.0,-0.392077,-0.971015,1,1
3,1.0,-0.461621,-0.294478,1,1
4,1.0,0.108246,-0.031380,0,1
5,1.0,0.011017,1.547205,1,1


In [22]:
# create the x (independent) and y (dependent) variables
# --------------------------------------------------------
y = loan_prep[['status_Y']]
x = loan_prep.drop(['status_Y'], axis=1)

In [26]:
# split the x and y dataset into training and test set
# -------------------------------------------------------
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = \
  train_test_split(x, y, test_size=0.30, random_state=1234, stratify=y) 

In [32]:
# apply the logistic regression model
# --------------------------------------------------------
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression()
lrc.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
# retrieve the predictions in 'status_Y'
y_predict = lrc.predict(x_test)
y_predict

array([0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1], dtype=uint8)

In [45]:
# evaluate the model with confusion matrix -- compare the predict and test sets
# ------------------------------------------------------------------------------
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
cm

array([[ 29,  20],
       [  2, 108]])

In [53]:
# score the model with accuracy 
acc = lrc.score(x_test, y_test)
acc.round(4)*100

86.16